### 4.5. Modelo Final

&emsp;Aqui apresentamos nossa versão do modelo preditivo para detecção de anomalias no consumo de gás. Esta seção do documento descreve detalhadamente o processo de construção do modelo, desde o tratamento e a preparação dos dados até a escolha do algoritmo mais adequado para a tarefa. Além disso, abordamos o procedimento de carregamento do arquivo salvo com o modelo treinado e fornecemos uma análise detalhada dos resultados obtidos, avaliando o desempenho do modelo com base em métricas relevantes e discutindo sua eficácia na identificação de padrões anômalos no consumo de gás.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.utils import resample
import pandas as pnd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

In [ ]:
listas = ['month_2.csv', 'month_3.csv', 'month_4.csv', 'month_5.csv', 'month_6.csv']
df = []
for arquivo in listas:
    df += [pnd.read_csv(arquivo)]
#Concatena todos os dataframes em um único dataframe chamado df
df = pnd.concat(df)

#### 4.5.1. Organização dos Dados

In [ ]:
#Chama o dataframe contido na variável chamada df
dadosCadastrais = pnd.read_csv('informacao_cadastral.csv')
usuariosUnicos = dadosCadastrais[dadosCadastrais.situacao == 'CONSUMINDO GÁS']['clientCode'].unique() 
#Organiza os dados dos usuários filtrados pela data
mesFiltrado = df[df['clientCode'].isin(usuariosUnicos)].sort_values(by='datetime') 
#Filtra meterSN diferente de '>N<A'
df = mesFiltrado[mesFiltrado['meterSN'] != '>N<A']
#Garante que todas as linhas com gain nulo sejam preenchidas com 1. Não é garantido que é o valor correto, mas é o melhor que podemos fazer
df['gain'].fillna(1, inplace=True)
#Corrige os pulsos para m²
df['pulseCount'] = df['pulseCount'] * df['gain']
#Cria a variação do pulseCount como uma coluna nova, calculando por grupo a diferença
df['datetime'] = pnd.to_datetime(df['datetime'])
df['dateTimeSegundos'] = df['datetime'].astype(np.int64) // 10**9
df['diffDateTime'] = df.groupby(['clientCode', 'meterSN']).dateTimeSegundos.diff()
df['diffPulseCount'] = df.groupby(['clientCode', 'meterSN']).pulseCount.diff()
df['diffPulseCountTempo'] = df['diffPulseCount'] / df['diffDateTime']
#Preenche os valores nulos (iniciais) com 0
df['diffDateTime'].fillna(0, inplace=True) 
df['diffPulseCount'].fillna(0, inplace=True)
df['diffPulseCountTempo'].fillna(0, inplace=True)
#Reseta o index
df.reset_index(drop=True, inplace=True)
#Seleciona as colunas que serão usadas
df = df[['clientCode', 'meterSN', "pulseCount", 'diffPulseCount','datetime', 'diffDateTime', 'diffPulseCountTempo', 'dateTimeSegundos']]
#Calcula a média e o desvio padrão do diffPulseCount por cliente
df['mediaCliente'] = df.groupby(['clientCode', 'meterSN']).diffPulseCount.transform('mean')
df['desvioPadraoCliente'] = df.groupby(['clientCode', 'meterSN']).diffPulseCount.transform('std')
df['diffDateTime'].describe()
df['mediaPCTCliente'] = df.groupby(['clientCode', 'meterSN']).diffPulseCountTempo.transform('mean')
df['desvioPadraoPCTCliente'] = df.groupby(['clientCode', 'meterSN']).diffPulseCountTempo.transform('std')
#Inicializa nova coluna de tipo e preenche com valor padrão 'c
df['tipo'] = "c"

In [ ]:
#Classifica os inidivíduos sem medições por longos períodos de tempo
df.loc[df['diffDateTime'] > 86400, 'tipo'] = "sm1"
df.loc[df['diffDateTime'] > 604800, 'tipo'] = "sm7"
df.loc[df['diffDateTime'] > 2592000, 'tipo'] = "sm30"

In [ ]:
#Classifica consumo acima de 3 desvio padrão, consumo negativo e consumo zerado
df.loc[df['diffPulseCountTempo'] > df['mediaPCTCliente'] + 3 * df['desvioPadraoPCTCliente'], 'tipo'] = "dp3"
df.loc[df['diffPulseCountTempo'] < 0, 'tipo'] = "cn"
df.loc[(df['pulseCount'] == 0) & (df['diffPulseCountTempo'] < 0), 'tipo'] = "cz"

#### 4.5.2. Modelagem Para o Problema

&emsp;Utilizamos o servidor da Inteli, equipado com um processador Intel Core i7-13700K e 32GB de RAM, para rodar nosso modelo preditivo. A escolha desse servidor foi motivada por sua alta capacidade de processamento, permitindo que as execuções fossem realizadas de maneira eficiente e rápida. Em relação ao modelo final específico que selecionamos, a decisão foi baseada nos resultados satisfatórios obtidos em termos de acurácia, precisão e recall. Esses indicadores demonstraram a eficácia do modelo na classificação correta das instâncias e no equilíbrio entre a identificação de verdadeiros positivos e a minimização de falsos positivos e falsos negativos.

In [ ]:
X = df[['diffPulseCountTempo', 'mediaPCTCliente', 'desvioPadraoPCTCliente', 'diffDateTime']]  # Features
y = df['tipo']  # Target
#Cria conjunto de teste e de treino
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#Tentativa de balancear as classes
X_majority = X_test[y_test == 'c']
X_minority = X_test[y_test != 'c']
y_majority = y_test[y_test == 'c']
y_minority = y_test[y_test != 'c']
X_majority_downsampled, y_majority_downsampled = resample(X_majority, y_majority, replace=False, n_samples=len(X_minority), random_state=42)
X_test_balanced = pnd.concat([X_majority_downsampled, X_minority])
y_test_balanced = pnd.concat([y_majority_downsampled, y_minority])

In [ ]:
# Rodando o Grid Search no conjunto de treino

model = RandomForestClassifier(random_state=42)

# Definindo os hiperparâmetros para o Grid Search
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Configurando o Grid Search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, 
                           cv=5, n_jobs=-1, verbose=2, scoring='accuracy')

# Rodando o Grid Search no conjunto de treino
grid_search.fit(X_train, y_train)

# Exibindo os melhores hiperparâmetros
print("Melhores parâmetros: ", grid_search.best_params_)

# Avaliando o modelo no conjunto de teste
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Avaliar a acurácia
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia no set de teste: {accuracy:.2f}")

&emsp; Com esse grid search, chegamos nos seguintes hiperparâmetros do modelo:

In [ ]:
# Definindo o modelo
model = RandomForestClassifier(
    random_state=42,
    max_depth=None,
    min_samples_leaf=1,
    min_samples_split=5,
    n_estimators=100,
    n_jobs=-1
)

In [ ]:
print("Prestes a fazer .fit do modelo")
model.fit(X_train, y_train)

In [ ]:
print("Prestes a fazer .predict do modelo")
# Avaliando o modelo no conjunto de teste
y_pred = model.predict(X_test_balanced)

print("O modelo rodou")

In [ ]:
# Avaliar a acurácia
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test_balanced, y_pred)
print(f"Acurácia nos dados de teste: {accuracy:.2f}")

print(classification_report(y_test_balanced, y_pred))

#### 4.5.3. Rodando o Modelo Salvo

Aqui executamos o modelo a partir de um arquivo previamente salvo, criado no servidor do Inteli:

In [ ]:
import joblib

# Load model with joblib
loaded_model = joblib.load('..\models\modelofinal.joblib')

In [ ]:
y_pred = loaded_model.predict(X_test_balanced)

In [ ]:
# Avaliar a acurácia
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test_balanced, y_pred)
print(f"Acurácia nos dados de teste: {accuracy:.2f}")

print(classification_report(y_test_balanced, y_pred))

#### 4.5.4. Métricas Relacionadas ao Modelo

##### 4.5.4.1. Acurácia

In [ ]:
# Avaliar a acurácia
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test_balanced, y_pred)
print(f"Acurácia nos dados de teste: {accuracy:.2f}")

&emsp;Considerando que nossa acurácia foi em torno de 0,97, podemos afirmar que atingimos uma acurácia satisfatória, o que significa que o modelo foi capaz de prever corretamente 97% das instâncias. Esse desempenho elevado reforça a capacidade do modelo para as previsões, mostrando-se confiável na maior parte dos casos.

##### 4.5.4.2. Precisão

In [ ]:
# Avaliar a precisão
from sklearn.metrics import precision_score
precision = precision_score(y_test_balanced, y_pred, average='weighted')
print(f"Precisão nos dados de teste: {precision:.2f}")

&emsp;A precisão do modelo variou entre 0,92 e 1, o que também representa um desempenho muito bom. Esse intervalo indica que, na maioria das vezes, o modelo conseguiu classificar corretamente as instâncias positivas, reduzindo o número de falsos positivos. Mesmo nos casos em que a precisão foi de 0,92, ainda assim o modelo mostrou uma boa capacidade de identificação correta, o que é crucial para garantir previsões confiáveis e tomadas de decisão mais assertivas com base nos resultados.

&emsp;É bom tomar em conta o macro average também, que deu 0,97, o que significa que o desempenho geral do modelo foi consistentemente elevado em todas as classes, sem favorecer uma classe específica. O macro average considera a média da precisão para cada classe, independentemente de sua proporção no conjunto de dados. Portanto, um valor alto como 0,97 indica que o modelo manteve uma boa performance em todas as classes, mesmo nas menos representadas, reforçando a robustez do modelo ao lidar com diferentes cenários de classificação.

##### 4.5.4.3. Recall

In [ ]:
# Avaliar o recall
from sklearn.metrics import recall_score
recall = recall_score(y_test_balanced, y_pred, average='weighted')
print(f"Recall nos dados de teste: {recall:.2f}")

&emsp;O "weighted recall" apresentou um valor de 0,97, o que indica que o modelo teve uma excelente capacidade de identificar corretamente as instâncias positivas em todas as classes, levando em consideração o peso de cada uma delas no conjunto de dados. Ao analisar o recall com base no número de exemplos em cada classe, o "weighted recall" garante que o desempenho do modelo nas classes maiores tenha mais impacto na métrica final, sem desconsiderar as menores. Esse resultado de 0,97 reflete uma alta taxa de acertos globais, com um equilíbrio eficiente na classificação correta das instâncias, o que é essencial para minimizar falsos negativos e aumentar a confiabilidade do modelo.

&emsp;Assim como o "weighted recall", a precisão do modelo também apresentou um excelente valor de 0,97, o que significa que, entre todas as instâncias classificadas como positivas, uma grande proporção foi corretamente identificada. Isso reflete a capacidade do modelo de minimizar falsos positivos, garantindo que os exemplos previstos como positivos sejam realmente relevantes. Em combinação com o recall, essa alta precisão também contribui para a minimização de falsos negativos, pois, ao identificar corretamente as instâncias, o modelo reduz a chance de deixar passar exemplos positivos não detectados. Esse equilíbrio entre precisão e recall é fundamental para assegurar a confiabilidade das predições do modelo e aumentar a confiança nas decisões baseadas nesses resultados.

#### 4.5.3. Modelo Candidato e Discussão Sobre os Resultados do Modelo.


&emsp;Optamos por este modelo final devido a três fatores principais. Primeiro, sua capacidade de classificar anomalias com precisão foi um diferencial em comparação a outras opções. Segundo, a qualidade dos labels disponíveis facilitou o treinamento e contribuiu para o bom desempenho do modelo. Por fim, a eficiência de implementação também foi um fator relevante, tornando-o uma escolha prática e eficaz para atender aos objetivos do projeto.

&emsp;Além de sua eficiência e capacidade de identificar anomalias, o modelo foi executado nos servidores do Inteli devido à alta demanda computacional necessária para processar o volume de dados e realizar as iterações com rapidez. A infraestrutura do servidor garantiu que o modelo rodasse de forma otimizada, permitindo realizar ajustes e análises em tempo hábil, o que tornou o processo de salvar o modelo mais eficiente.

&emsp;No geral, os resultados alcançados pelo modelo foram bastante satisfatórios. Quando avaliamos o desempenho com base em diferentes métricas, como a acurácia de 0,97, a precisão de 0,97 e o recall de 0,97, fica claro que o modelo conseguiu um equilíbrio excelente entre identificar corretamente as instâncias positivas e minimizar erros. Essas métricas indicam uma alta capacidade de generalização, o que é fundamental para garantir a confiabilidade das previsões e suportar decisões estratégicas com segurança. Além disso, o fato de essas métricas não atingirem 100% é, na verdade, um ponto positivo, pois sugere que o modelo não sofreu de overfitting, mantendo uma performance favorável tanto nos dados de treino quanto nos dados de teste.

#### 4.5.4. Plano de Contingência

&emsp;O plano de contingência para casos em que o modelo falhe, como a ocorrência de falsos negativos, será evitar a utilização dos resultados do modelo nessas situações, garantindo que eles não sejam considerados automaticamente como corretos. Quando o modelo deixar de identificar uma anomalia que existe de fato, os resultados serão suspensos e encaminhados para uma análise profunda e manual por especialistas antes de qualquer ação ou decisão ser tomada. Essa abordagem preventiva assegura que possíveis falhas no modelo não comprometam a integridade das operações. Somente após essa revisão cuidadosa os resultados poderão ser validados, minimizando riscos e assegurando que erros de predição não afetem negativamente o processo de tomada de decisão.

#### 4.5.5. Conclusão

&emsp;Avaliamos os resultados do modelo em relação às personas e ao entendimento de negócio. O modelo atendeu às dores de Cintia, proporcionando uma ferramenta eficaz para identificar anomalias com precisão e criar visualizações claras e impactantes, melhorando a comunicação com seus superiores. Além disso, o modelo também foi fundamental para atender às necessidades de Pedro, garantindo que todos os membros da equipe pudessem entender os dados e utilizá-los em suas respectivas áreas, facilitando o processo de análise antes de qualquer decisão importante. Ele se alinha perfeitamente ao entendimento de negócio, facilitando a detecção rápida de anomalias e agilizando o processo de correção, o que reduz o tempo de resposta e potencializa a eficiência operacional. Ao mesmo tempo, o modelo diferencia a empresa no mercado, oferecendo uma solução inovadora e robusta, destacando-a frente aos concorrentes e dando a Pedro a confiança necessária para tomar decisões estratégicas com base em dados concretos e precisos.

&emsp;Considerando o nosso modelo de negócio, o modelo desenvolvido não apenas facilita a identificação de anomalias com alta precisão, como também agiliza o processo de correção, permitindo que problemas sejam resolvidos de forma rápida e eficiente. Isso gera um impacto direto nos resultados do negócio, uma vez que a rapidez na detecção e correção de anomalias minimiza riscos e otimiza a operação. Além disso, ao oferecer uma solução diferenciada no mercado, a empresa se destaca em relação aos concorrentes, consolidando sua capacidade de entregar previsões confiáveis, o que fortalece sua posição competitiva e agrega valor estratégico.

&emsp;A implementação deste modelo atende diretamente às necessidades de personas como a Cintia, que enfrenta desafios na previsão de anomalias e na criação de visualizações de dados impactantes. A capacidade do modelo de classificar anomalias com alta precisão (acurácia, precisão e recall de 0,97) fornece uma ferramenta completa para a identificação de padrões críticos que antes passavam despercebidos. Além disso, os resultados confiáveis e generalizáveis permitem que Cintia apresente dados com maior clareza e segurança para seus superiores, facilitando a comunicação e a tomada de decisões estratégicas. A facilidade de ajuste e a eficiência na execução do modelo também contribuem para que ela mantenha o controle do processo, economizando tempo e recursos valiosos.

&emsp;A conclusão deste projeto reflete como o modelo atende diretamente às necessidades da persona Pedro, ao facilitar a compreensão dos dados por todos os membros da sua equipe de desenvolvimento. A precisão na classificação de anomalias e a qualidade dos labels proporcionam uma base sólida para que cada membro entenda as informações mais relevantes para suas respectivas áreas, promovendo decisões mais informadas e colaborativas. Além disso, o modelo, com seu desempenho eficiente, oferece a Pedro uma ferramenta sólida para lidar com grandes volumes de dados. Ele pode visualizar e analisar cautelosamente as informações, garantindo que cada decisão sobre a implementação do produto seja fundamentada em dados sólidos e consistentes, sem comprometer a agilidade do processo. Isso assegura que ele mantenha o controle e a confiança em suas decisões, alinhando a eficácia técnica às demandas estratégicas do projeto.